In [3]:
import plotly.graph_objects as go

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import numpy as np
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep

from datetime import datetime
from datetime import timedelta
import os
import time
from plotly.subplots import make_subplots
import glob

## 데이터 수집

In [52]:
ticker_nm = '005930'
ticker_nm = '247540'
start_date  = '20190101'
today_date1 = '20231020'

df_raw = stock.get_market_ohlcv(start_date, today_date1, ticker_nm)
df_raw = df_raw.reset_index()
df_raw['ticker'] = ticker_nm

df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume','price_change_percentage', 'ticker']

## 보조지표 계산

### 이동평균선 계산

In [53]:
df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

### 볼린저밴드 계산

In [54]:
std = df_raw['close'].rolling(20).std(ddof=0)

df_raw['upper'] = df_raw['MA20'] + 2 * std
df_raw['lower'] = df_raw['MA20'] - 2 * std

### MACD 계산

In [55]:
# MACD 
macd = MACD(close=df_raw['close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)


df_raw['MACD_DIFF'] = macd.macd_diff()
df_raw['MACD'] = macd.macd()
df_raw['MACD_Signal'] = macd.macd_signal()

### RSI 계산

In [56]:
df_raw['변화량'] = df_raw['close'] - df_raw['close'].shift(1)
df_raw['상승폭'] = np.where(df_raw['변화량']>=0, df_raw['변화량'], 0)
df_raw['하락폭'] = np.where(df_raw['변화량'] <0, df_raw['변화량'].abs(), 0)

# welles moving average
df_raw['AU'] = df_raw['상승폭'].ewm(alpha=1/14, min_periods=14).mean()
df_raw['AD'] = df_raw['하락폭'].ewm(alpha=1/14, min_periods=14).mean()
df_raw['RSI'] = df_raw['AU'] / (df_raw['AU'] + df_raw['AD']) * 100

### 이동평균선 차이 계산

In [57]:
df_raw['MA5-20'] = df_raw['MA5'] - df_raw['MA20']
df_raw['MA20-60'] = df_raw['MA20'] - df_raw['MA60']
df_raw['MA60-120'] = df_raw['MA60'] - df_raw['MA120']

## 보조지표 분석

In [58]:
df_raw_anal = df_raw[['date','ticker', 'close']]

### 골든크로스

In [59]:
# 골든 크로스 5-20
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_5_20_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] > 0 and df_raw["MA5-20"].iloc[idx - 1] <= 0]

# 데드 크로스 5-20
# 양수에서 음수로 바뀌는 모든 인덱스 찾기
idx_5_20_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA5-20"].iloc[idx] < 0 and df_raw["MA5-20"].iloc[idx - 1] >= 0]

# 골든 크로스 20-60
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_20_60_gold_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] > 0 and df_raw["MA20-60"].iloc[idx - 1] <= 0]

# 골든 크로스 20-60
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_20_60_dead_cross = [idx for idx in range(len(df_raw)) if df_raw["MA20-60"].iloc[idx] < 0 and df_raw["MA20-60"].iloc[idx - 1] >= 0]

In [60]:
df_raw_anal.loc[:, '5_20_cross'] = '-'
df_raw_anal.loc[idx_5_20_gold_cross,'5_20_cross'] = '골든크로스(매수)'
df_raw_anal.loc[idx_5_20_dead_cross,'5_20_cross'] = '데드크로스(매도)'

df_raw_anal.loc[:, '20_60_cross'] = '-'
df_raw_anal.loc[idx_20_60_gold_cross,'20_60_cross'] = '골든크로스(매수)'
df_raw_anal.loc[idx_20_60_dead_cross,'20_60_cross'] = '데드크로스(매도)'

/tmp/ipykernel_1629324/1314038236.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_1629324/1314038236.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### 정배열, 역배열

In [61]:
ascending_sq  = (df_raw['MA5-20'] > 0) & \
(df_raw['MA20-60'] > 0) & \
(df_raw['MA60-120'] > 0) 

descending_sq  = (df_raw['MA5-20'] < 0) & \
(df_raw['MA20-60'] < 0) & \
(df_raw['MA60-120'] < 0) 

In [62]:
df_raw_anal.loc[:,'array'] = '-'
df_raw_anal.loc[ascending_sq,'array'] = '정배열(매수)'
df_raw_anal.loc[descending_sq,'array'] = '역배열(매도)'

/tmp/ipykernel_1629324/2497960516.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### 볼린저밴드

In [63]:
down_reg_sq = df_raw['upper'] - df_raw['close'] 
top_reg_sq  = df_raw['lower'] - df_raw['close'] 

down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
top_reg = [idx for idx in range(1,len(df_raw)) if top_reg_sq[idx] < 0 and top_reg_sq[idx-1] >= 0]

In [64]:
df_raw_anal.loc[:,'Bollinger_band'] = '-'
df_raw_anal.loc[ascending_sq,'Bollinger_band'] = '하향회귀(매도)'
df_raw_anal.loc[descending_sq,'Bollinger_band'] = '상향회귀(매수)'

/tmp/ipykernel_1629324/2605943495.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### MACD

In [65]:
signal_down_cross = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] < 0 and df_raw['MACD_DIFF'][idx-1] >= 0]
signal_top_corss = [idx for idx in range(1,len(df_raw)) if df_raw['MACD_DIFF'][idx] > 0 and df_raw['MACD_DIFF'][idx-1] <= 0]


In [66]:
df_raw_anal.loc[:,'MACD'] = '-'
df_raw_anal.loc[signal_down_cross,'MACD'] = '하향돌파(매도)'
df_raw_anal.loc[signal_top_corss,'MACD'] = '상향돌파(매수)'

/tmp/ipykernel_1629324/2822066516.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### RSI

In [67]:
down_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] > 70 and df_raw['RSI'][idx-1] <= 70]
top_reg = [idx for idx in range(1,len(df_raw)) if df_raw['RSI'][idx] < 30 and df_raw['RSI'][idx-1] >= 30]


In [68]:
df_raw_anal.loc[:,'RSI'] = '-'
df_raw_anal.loc[down_reg,'RSI'] = 'RSI 상단 하향돌파(매도)'
df_raw_anal.loc[top_reg,'RSI'] = 'RSI 하단 상향 돌파(매수)'

/tmp/ipykernel_1629324/819055055.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## 날짜

In [69]:
date = '2023-08-01'

In [70]:
df_raw_anal_date = df_raw_anal[df_raw_anal['date'] > date].reset_index(drop = True)
df_raw_date = df_raw[df_raw['date'] > date].reset_index(drop = True)

In [71]:
df_raw_anal_date.head(7)

,date,ticker,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2023-08-02,247540,380500,-,-,정배열(매수),하향회귀(매도),-,-
1,2023-08-03,247540,390000,-,-,정배열(매수),하향회귀(매도),하향돌파(매도),-
2,2023-08-04,247540,380500,-,-,정배열(매수),하향회귀(매도),-,-
3,2023-08-07,247540,340000,-,-,정배열(매수),하향회귀(매도),-,-
4,2023-08-08,247540,339000,-,-,정배열(매수),하향회귀(매도),-,-
5,2023-08-09,247540,343500,데드크로스(매도),-,-,-,-,-
6,2023-08-10,247540,337500,-,-,-,-,-,-


In [79]:
type_list = ['매수', '매도']

In [82]:
technical_indicator = ['5_20_cross', '20_60_cross', 'array', 'Bollinger_band', 'MACD', 'RSI']

In [83]:
type_nm = '매수'
technical_indicator_nm = '5_20_cross'

In [88]:
for type_nm in type_list:
    for technical_indicator_nm in technical_indicator:
#         print(df_raw_anal_date.index[df_raw_anal_date[technical_indicator_nm].str.contains(type_nm)])
        index_list = df_raw_anal_date.index[df_raw_anal_date[technical_indicator_nm].str.contains(type_nm)]
        for index_nm in index_list:
            print(index_nm)
        

0
1
2
3
4
36
46
40
5
26
0
1
2
3
4
1
37


In [86]:
df_raw_anal_date['']

,date,ticker,close,5_20_cross,20_60_cross,array,Bollinger_band,MACD,RSI
0,2023-08-02,247540,380500,-,-,정배열(매수),하향회귀(매도),-,-
1,2023-08-03,247540,390000,-,-,정배열(매수),하향회귀(매도),하향돌파(매도),-
2,2023-08-04,247540,380500,-,-,정배열(매수),하향회귀(매도),-,-
3,2023-08-07,247540,340000,-,-,정배열(매수),하향회귀(매도),-,-
4,2023-08-08,247540,339000,-,-,정배열(매수),하향회귀(매도),-,-
5,2023-08-09,247540,343500,데드크로스(매도),-,-,-,-,-
6,2023-08-10,247540,337500,-,-,-,-,-,-
7,2023-08-11,247540,323500,-,-,-,-,-,-
8,2023-08-14,247540,318000,-,-,-,-,-,-
9,2023-08-16,247540,303500,-,-,-,-,-,-


In [76]:
df_raw_anal_date.index[df_raw_anal_date['5_20_cross'] == '데드크로스(매도)']

Index([5], dtype='int64')

In [72]:
asdf = df_raw_anal_date.index[df_raw_anal_date['20_60_cross'].str.contains('매도')]
for cross_index in asdf:
    cross_date = df_raw['date'][cross_index]
    cross_value = df_raw['close'][cross_index]

    fig.add_annotation(x=cross_date, 
                       y=cross_value,
                       text=cross_name,
                       showarrow=True,
                       arrowhead=1,
                       row = 1, col = 1)

NameError: name 'fig' is not defined

In [73]:
# 상향, 하향 회귀
for i in range(len(cross_df)):
    cross_index = cross_df['index'][i]
    cross_name = cross_df['name'][i]

    cross_date = df_raw['date'][cross_index]
    cross_value = df_raw['close'][cross_index]

    fig.add_annotation(x=cross_date, 
                       y=cross_value,
                       text=cross_name,
                       showarrow=True,
                       arrowhead=1,
                       row = 1, col = 1)

NameError: name 'cross_df' is not defined

## 시각화

In [98]:
macd_vis(df_raw_date)    

/home/shjj08choi/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [97]:
def macd_vis(df_raw):
    # fig = go.Figure()
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.5, 0.1, 0.2, 0.2])

    # 캔들스틱차트
    fig.add_trace(go.Candlestick(
        x=df_raw['date'],
        open=df_raw['open'],
        high=df_raw['high'],
        low=df_raw['low'],
        close=df_raw['close'],
        increasing_line_color= 'red', decreasing_line_color= 'blue', 
        name = ''), row = 1, col = 1)

    # MA 5 
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA5'],
                             opacity=0.7,
                             line=dict(color='blue', width=2),
                             name='MA 5') , row = 1, col = 1)

    # MA 20
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA20'],
                             opacity=0.7,
                             line=dict(color='orange', width=2),
                             name='MA 20'), row = 1, col = 1)

    # MA 60
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA60'],
                             opacity=0.7,
                             line=dict(color='purple', width=2),
                             name='MA 60'), row = 1, col = 1)

    # MA 120
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA120'],
                             opacity=0.7,
                             line=dict(color='green', width=2),
                             name='MA 120'), row = 1, col = 1)

    fig.add_trace(go.Scatter(
        x=pd.concat([df_raw['date'], df_raw['date'][::-1]]),
        y=pd.concat([df_raw['upper'], df_raw['lower'][::-1]]),
        fill='toself',
        fillcolor='rgba(255,255,0,0.1)',
        line=dict(color='rgba(255,255,255,0.2)', width=2),
        name='Bollinger Band',
        showlegend=False
    ), row = 1, col = 1)

    for type_nm in type_list:
        for technical_indicator_nm in technical_indicator:
    #         print(df_raw_anal_date.index[df_raw_anal_date[technical_indicator_nm].str.contains(type_nm)])
            index_list = df_raw_anal_date.index[df_raw_anal_date[technical_indicator_nm].str.contains(type_nm)]
            for index_nm in index_list:
                cross_date = df_raw['date'][index_nm]
                cross_value = df_raw['close'][index_nm]
                
                fig.add_annotation(x=cross_date, 
                                   y=cross_value,
                                   text=f'{type_nm} <br> {technical_indicator_nm}',
                                   showarrow=True,
                                   arrowhead=1,
                                   row = 1, col = 1)                
            
#     asdf = df_raw_anal_date.index[df_raw_anal_date['20_60_cross'].str.contains('매도')]
#     for cross_index in asdf:
#         cross_date = df_raw['date'][cross_index]
#         cross_value = df_raw['close'][cross_index]

#         fig.add_annotation(x=cross_date, 
#                            y=cross_value,
#                            text='cross_name',
#                            showarrow=True,
#                            arrowhead=1,
#                            row = 1, col = 1)

    # Row 2 
    # volume
    colors = ['blue' if row['open'] - row['close'] >= 0 
              else 'red' for index, row in df_raw.iterrows()]

    fig.add_trace(go.Bar(x=df_raw['date'], 
                         y=df_raw['volume'],
                         marker_color=colors,
                         name = 'Volume',
                         showlegend=False
                        ), row=2, col=1)

    # MACD
    colors = ['red' if val >= 0 
              else 'blue' for val in df_raw['MACD_DIFF']]
    fig.add_trace(go.Bar(x=df_raw['date'], 
                         y=df_raw['MACD_DIFF'],
                         marker_color=colors,
                         name =  'MACD DIFF'
                        ), row=3, col=1)

    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['MACD'],
                             line=dict(color='green', width=2),
                             name = 'MACD'
                            ), row=3, col=1)
    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['MACD_Signal'],
                             line=dict(color='orange', width=1),
                             name = 'MACD Signal'
                            ), row=3, col=1)

    # Row 5
    # RSI
    fig.add_trace(go.Scatter(x=df_raw['date'],
                             y=df_raw['RSI'],
                             line=dict(color='red', width=1),
                             name = 'RSI'
                            ), row=4, col=1)
    
    # 수평 사각 영역 추가하기
    
    fig.add_hrect(y0=70, y1=100, line_width=0, fillcolor="white", opacity=0.1,
                  annotation_text="과매수구간", 
                  annotation_position="top right",
                  annotation_font_size=10,
                  annotation_font_color="red",
                  annotation_font_family="Times New Roman", row=4, col=1)
    
    fig.add_hrect(y0=0, y1=30, line_width=0, fillcolor="white", opacity=0.1,
                  annotation_text="과매도 구간", 
                  annotation_position="top right",
                  annotation_font_size=10,
                  annotation_font_color="blue",
                  annotation_font_family="Times New Roman", row=4, col=1)    
    
    
    # Rayout
    fig.update_layout(
        title = '삼성전자 주가',
        title_font_family="맑은고딕",
        title_font_size = 18,
        hoverlabel=dict(
            bgcolor='black',
            font_size=15,
        ),
        hovermode="x unified",
        template='plotly_dark',
        xaxis_tickangle=90,
        yaxis_tickformat = ',',
        legend = dict(orientation = 'h', xanchor = "center", x = 0.5, y= 1.2),
        barmode='group',
        margin=go.layout.Margin(
            l=10, #left margin
            r=10, #right margin
            b=10, #bottom margin
            t=100  #top margin
        ),
        height=400, width=900, 
    #     showlegend=False, 
        xaxis_rangeslider_visible=False
    )

    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="MACD", row=3, col=1)
    fig.update_yaxes(title_text="RSI", row=4, col=1)

    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    return fig
